# ADA - Project
## With Spark

## Getting started

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext("local", "ADA")

In [3]:
data = sc.parallelize(range(100))
data.sum()

4950

If you received 4950 as a result, your spark is working well :) Good job !

## Configuration

In [4]:
config = {
    "session":"progfun-002"
}

## Importing and parsing the data

In [5]:
# Reading csv files: Create RDD () with one string entry per line in the file
rdd_problem_events = sc.textFile("../data/"+config['session']+"_Problem_Events_with_Info.csv")
rdd_video_events = sc.textFile("../data/"+config['session']+"_Video_Events.csv")
rdd_forum_events = sc.textFile("../data/"+config['session']+"_Forum_Events.csv")

# Prints the first line (header) along with indices for each table
print("--- PROBLEM_EVENTS ---")
for idx,field in enumerate(rdd_problem_events.take(1)[0].split(",")): 
    print(idx,field, end=" / ")

print("\n\n--- VIDEO_EVENTS ---")
for idx,field in enumerate(rdd_video_events.take(1)[0].split(",")): 
    print(idx,field, end=" / ")

print("\n\n--- FORUM_EVENTS ---")
for idx,field in enumerate(rdd_forum_events.take(1)[0].split(",")): 
    print(idx,field, end=" / ")

--- PROBLEM_EVENTS ---
0 EventID / 1 ForumUserID / 2 MaximumSubmissions / 3 AccountUserID / 4 SubmissionNumber / 5 Grade / 6 TimeStamp / 7 DataPackageID / 8 ProblemID / 9 SoftCloseTime / 10 ProblemType / 11 HardCloseTime / 12 Platform / 13 OpenTime / 14 EventType / 15 Title / 16 SessionUserID / 17 UniqueProblemID / 18 UniqueUserID / 

--- VIDEO_EVENTS ---
0 EventID / 1 ForumUserID / 2 OldTime / 3 AccountUserID / 4 CurrentTime / 5 SeekType / 6 TimeStamp / 7 DataPackageID / 8 UniqueRowID / 9 TableName / 10 VideoID / 11 Platform / 12 NewSpeed / 13 EventSource / 14 EventType / 15 SessionUserID / 16 NewTime / 17 OldSpeed / 

--- FORUM_EVENTS ---
0 EventID / 1 ForumUserID / 2 PostID / 3 AccountUserID / 4 TimeStamp / 5 DataPackageID / 6 UniqueRowID / 7 TableName / 8 Platform / 9 EventSource / 10 PostType / 11 EventType / 12 JoinID / 13 SessionUserID / 

In [6]:
# Use the filter method to remove the first line
rdd_problem_events = rdd_problem_events.filter(lambda x: not x.startswith('EventID'))
rdd_video_events = rdd_video_events.filter(lambda x: not x.startswith('EventID'))
rdd_forum_events = rdd_forum_events.filter(lambda x: not x.startswith('EventID'))

# Prints first record for each table
print(rdd_problem_events.take(1)[0],"\n") 
print(rdd_video_events.take(1)[0],"\n") 
print(rdd_forum_events.take(1)[0],"\n")

3bcd1a54ed6ddb04b4a6fb2906110a01,None,0,None,7,None,1365344171,progfun-002,7,2147483640,Video,2147483640,Coursera,32400,Problem.Check,Lecture 1.2 - Elements of Programming (14:25),c4d4e5fcd2feba9f3234ee8d852dc7b22fbc07e4,f322944718b2ee0e53292118111533c7,21f13b3f6b50a83343b57d2f1d07dbdf 

db75adce6b87e7ab79242ea0af4b82d4,None,154.696,None,154.697,None,1372391638,progfun-002,00000078c0f0685cc50a25a8d5734a88,Video_Events,33,coursera,1.0,None,Video.Play,ef64fb7b096008f7eaf8441684afdf99af9af54a,None,1.0 

f3fdb52859b2511308aee554a573194e,None,17,4108315,1376254235,progfun-002,000006c12322ca29c7013dac42ef1a6a,Forum_Events,coursera,None,Thread,Forum.Thread.View,03b1fa287de5ef57d9c8482195b5167f,None 



In [7]:
# Function to parse the string entries from previous dataset
def parse_problems(x):
    data = x.split(',')
    return {
        'Grade':data[5],
        'TimeStamp':data[6],
        'ProblemID':data[8],
        'ProblemType':data[10],
        'EventType':data[14].split('.')[0],
        'EventSubType':data[14].split('.')[1],
        'SessionUserID':data[16]
    }

def parse_videos(x):
    data = x.split(',')
    return {
        'TimeStamp':data[6],
        'VideoID':data[10],
        'EventType':data[14].split('.')[0],
        'EventSubType':data[14].split('.')[1],
        'SessionUserID':data[15]
    }

def parse_forums(x):
    data = x.split(',')
    return {
        'AccountUserID':data[3],
        'TimeStamp':data[4],
        'EventType':data[11].split('.')[0],
        'EventSubType':data[11].split('.')[1]
    }

# Use the map method to have more workable data
rdd_problem_events_parsed = rdd_problem_events.map(parse_problems)
rdd_video_events_parsed = rdd_video_events.map(parse_videos)
rdd_forum_events_parsed = rdd_forum_events.map(parse_forums)

# Prints the count of elements along with the first element of each table
print(rdd_problem_events_parsed.count())
print(rdd_problem_events_parsed.take(1)[0],"\n") 
print(rdd_video_events_parsed.count()) 
print(rdd_video_events_parsed.take(1)[0],"\n") 
print(rdd_forum_events_parsed.count()) 
print(rdd_forum_events_parsed.take(1)[0],"\n") 


458888
{'Grade': 'None', 'EventSubType': 'Check', 'TimeStamp': '1365344171', 'EventType': 'Problem', 'SessionUserID': 'c4d4e5fcd2feba9f3234ee8d852dc7b22fbc07e4', 'ProblemID': '7', 'ProblemType': 'Video'} 

3471666
{'EventType': 'Video', 'VideoID': '33', 'EventSubType': 'Play', 'SessionUserID': 'ef64fb7b096008f7eaf8441684afdf99af9af54a', 'TimeStamp': '1372391638'} 

297650
{'EventSubType': 'Thread', 'AccountUserID': '4108315', 'EventType': 'Forum', 'TimeStamp': '1376254235'} 



In [8]:
# Handles problem with the forum events table having 'AccountUserID' instead of 'SessionUserID'
# Using the table progfun-002_User_Hash_Mapping
rdd_user_mapping = sc.textFile("../data/"+config['session']+"_User_Hash_Mapping.csv")
print(rdd_user_mapping.take(1)[0],"\n")

def f(x):
    x[1][1]['SessionUserID']=x[1][0] 
    return x[1][1]

rdd_forum_events_parsed = (rdd_user_mapping
    # removes header
    .filter(lambda x: not x.startswith("ForumUserID"))
    # maps to have the format (Key=AccountUserID,Value=SessionUserID)
    .map(lambda x:(x.split(",")[1],x.split(",")[6]))
    # join with rdd_forum_event to get format (map to have the (Key=AccountUserID,Value=(SessionUserID,EventObject)) format)
    .join(rdd_forum_events_parsed
        # map to have the (Key=AccountUserID,Value=EventObject) format
        .map(lambda x: (x['AccountUserID'],x))
    )
    # Use the function f to update EventObject with the joined SessionUserID
    .map(f)
)

print(rdd_forum_events_parsed.count())
print(rdd_forum_events_parsed.take(1)[0])

ForumUserID,AccountUserID,DataPackageID,UniqueRowID,TableName,Platform,SessionUserID 

297650
{'EventSubType': 'ThreadSubscribe', 'AccountUserID': '1932792', 'EventType': 'Forum', 'SessionUserID': 'a97848d806f0d88cf80dd154845693af40cf559f', 'TimeStamp': '1364416322'}


In [9]:
# Filters only Problem events of type Assignment
rdd_problem_events_parsed = rdd_problem_events_parsed.filter(lambda x: x['ProblemType']=='Assignment')
print(rdd_problem_events_parsed.count())
print(rdd_problem_events_parsed.take(1)[0])

105270
{'Grade': '9.32999992371', 'EventSubType': 'Check', 'TimeStamp': '1366220878', 'EventType': 'Problem', 'SessionUserID': 'd8f79efa32a560b8a46ea2b12d9bed97c9e39b4b', 'ProblemID': '6', 'ProblemType': 'Assignment'}


In [10]:
# Concatenantes all three table into one big table
rdd_events = (rdd_problem_events_parsed
    .union(rdd_video_events_parsed)
    .union(rdd_forum_events_parsed)
)
rdd_events.persist()
print(rdd_events.count())
print(rdd_events.take(1)[0])

3874586
{'EventSubType': 'Check', 'Grade': '9.32999992371', 'EventType': 'Problem', 'TimeStamp': '1366220878', 'ProblemType': 'Assignment', 'ProblemID': '6', 'SessionUserID': 'd8f79efa32a560b8a46ea2b12d9bed97c9e39b4b'}


## Working the data

### Grouping & Sorting

In [11]:
# uses the function groupByKey on our events with the key 'SessionStudentID'
rdd_events_by_students = (rdd_events
    .map(lambda x: (x['SessionUserID'],x))
    .groupByKey()
)
rdd_events_by_students.persist()
print(rdd_events_by_students.count()) # This is our number of students
print(rdd_events_by_students.take(1)[0])

31269
('75046568bfb19ce33846e4b7c384db7a8809d578', <pyspark.resultiterable.ResultIterable object at 0x1098da588>)


In [12]:
# Function to have a friendly way to print the events
def eventToString(event):
    return {
        "Problem": lambda x:"(P "+event['ProblemID']+" "+event['Grade']+")",
        "Video": lambda x:"(V "+event['VideoID']+")",
        "Forum": lambda x:"(F)",
    }[event['EventType']](event)

In [13]:
# Looks at the data for a few students
for studentID,events in rdd_events_by_students.take(3):
    print("\n",studentID,"\n")
    for event in events:
        print(eventToString(event), end=" ")
    print()


 75046568bfb19ce33846e4b7c384db7a8809d578 

(V 6) (V 3) (V 27) (V 49) (V 47) (V 25) (V 29) (V 4) (V 39) (V 41) (V 5) (V 43) (V 33) (V 35) (V 37) (V 7) (V 8) 

 3a3aea293c5a658c4d2cc8c0aaf07694b985bbec 

(V 79) (V 101) (V 73) (V 89) (V 85) (V 8) (V 123) (V 127) (V 47) (V 23) (V 105) (V 4) (V 41) (V 23) (V 43) (V 29) (V 29) (V 119) (V 49) (V 125) (V 33) (V 21) (V 109) (V 113) (V 111) (V 75) (V 3) (V 51) (V 27) (V 81) (V 87) (V 27) (V 117) (V 7) (V 27) (V 71) (V 25) (V 77) (V 37) (V 95) (V 103) (V 107) (V 115) (V 91) (V 39) (V 35) (V 83) (V 121) (V 93) (V 97) (V 6) (V 5) (V 53) 

 03ca065880520199acd8dbd9c743212cf0ba8861 

(V 21) (V 27) (V 5) (V 7) (V 3) (V 3) (V 5) (V 3) (V 21) (V 5) (V 3) (V 4) (V 25) (V 23) (V 5) (V 8) (V 5) (V 33) (V 23) (V 3) (V 6) (V 4) (V 29) (V 6) (V 4) (V 21) (V 21) (V 21) (V 23) (V 27) (V 6) (V 27) (V 8) (V 3) (V 6) (V 8) (V 27) (V 6) (V 33) (V 25) (V 21) (V 29) (V 25) (V 5) (V 3) (V 3) (V 21) (V 27) (V 6) (V 23) (V 29) (V 21) (V 6) (V 3) (V 23) (V 3) (V 3) (V 

In [14]:
# Uses the map function to sort the events in each student activity list 
rdd_events_by_students_sorted = (rdd_events_by_students
    .map(lambda x: (x[0],sorted(x[1], key=(lambda event: event['TimeStamp']))))
)
print(rdd_events_by_students_sorted.count())
print((lambda x: (x[0],[y['EventType'] for y in x[1]]))(rdd_events_by_students_sorted.take(1)[0]))

31269
('75046568bfb19ce33846e4b7c384db7a8809d578', ['Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video', 'Video'])


In [15]:
# Looks at the data for a student, now that it's sorted by TimeStamps
for studentID,events in rdd_events_by_students_sorted.filter(lambda x:x[0]=='6ea6949ca133acede360d3573f9d1168b3d70b51').take(1):
    print("\n",studentID,"\n")
    for event in events:
        print(eventToString(event), end=" ")
    print()


 6ea6949ca133acede360d3573f9d1168b3d70b51 

(V 27) (V 27) (V 29) (V 29) (V 8) (V 3) (V 3) (P 5 7.32999992371) (V 4) (P 5 10.0) (V 5) (V 5) (V 6) (V 6) (V 6) (V 6) (V 6) (V 6) (V 6) (V 6) (V 6) (V 6) (V 6) (V 6) (V 7) (V 7) (V 7) (V 7) (V 33) (V 33) (V 33) (V 33) (P 7 9.92000007629) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 39) (V 39) (V 39) (V 39) (V 39) (V 39) (V 39) (V 39) (V 41) (V 41) (V 41) (V 41) (V 43) (V 47) (V 49) (P 6 9.67000007629) (P 6 9.32999992371) (P 6 10.0) (V 51) (V 51) (V 51) (V 51) (V 51) (V 51) (V 51) (V 53) (V 53) (V 53) (V 53) (V 53) (V 53) (V 53) (V 53) (V 53) (V 75) (V 75) (V 75) (V 75) (V 75) (V 75) (V 75) (V 75) (V 75) (V 75) (V 75) (V 75) (P 12 9.22999954224) (V 79) (V 77) (V 81) (V 83) (V 85) (V 87) (V 71) (P 14 4.5) (P 14 5.09000015259) (V 

### Extracting learning patterns
The goal is to extract what a students does between a failed attempt at an assignment and a successful attempt

In [16]:
# The goal is to extract the list of event in between the first and last attempts at each problem
def extractPatterns(events):
    problems = {}
    # for each problem measure first and last entry
    for i,event in enumerate(events):
        if event['EventType']=='Problem':
            problems[event['ProblemID']]=problems.get(event['ProblemID'],{'min':i})
            problems[event['ProblemID']]['max']=i
    patterns = {}
    for problem in problems.keys():
        patterns[problem]=events[problems[problem]['min']:(1+problems[problem]['max'])] 
    return patterns
    

In [17]:
rdd_events_by_students_extract = (rdd_events_by_students_sorted
    .map(lambda x: (x[0],extractPatterns(x[1])))
)

rdd_events_by_students_extract.persist()

DEBUG_STUDENTS=[
    '6ea6949ca133acede360d3573f9d1168b3d70b51',
    '5046ee71bc77983a0753d6e5ba98f5f1c685072e',
    '3b305429f93de02637949578a5f9e23f13eb0726',
    '9625e050ef2326c12632c51aea7b5e49a20d6fc7',
    '865981d5b40a693bafbadae4b1df769be03a25c3',
    '67cdae6073d1089b695e2a615a01187586ad7ba6',
    'fb4c81b3df430d1f0fbb8d0ca3e470ac6bf92a2f'
]

# Looks at the data for a student, now that it's sorted by TimeStamps
def displayStudentsPatterns(rdd,students):
    for studentID,patterns in rdd.filter(lambda x:x[0] in students).take(len(students)):
        print(studentID)
        for pattern in patterns.values():
            for event in pattern:
                print(eventToString(event), end=" ")
            print()
        print()

displayStudentsPatterns(rdd_events_by_students_extract,DEBUG_STUDENTS)

6ea6949ca133acede360d3573f9d1168b3d70b51
(P 14 4.5) (P 14 5.09000015259) (V 73) (V 89) (V 91) (V 93) (V 95) (V 95) (V 97) (V 101) (P 14 6.88000011444) (P 14 8.06999969482) (P 14 9.55000019073) (P 14 9.55000019073) (P 14 9.55000019073) (P 14 10.0) (P 14 10.0) 
(P 17 5.57000017166) 
(P 5 7.32999992371) (V 4) (P 5 10.0) 
(P 12 9.22999954224) 
(P 7 9.92000007629) 
(P 20 5.82999992371) (F) (F) (V 125) (V 125) (P 20 10.0) 
(P 6 9.67000007629) (P 6 9.32999992371) (P 6 10.0) 

67cdae6073d1089b695e2a615a01187586ad7ba6
(P 14 10.0) 
(P 17 10.0) 
(P 12 8.55000019073) (P 12 8.55000019073) (V 79) (V 77) (V 81) (V 83) (V 85) (V 83) (V 85) (V 87) (V 87) (V 71) (V 71) (V 79) (V 77) (V 81) (V 85) (V 87) (P 14 10.0) (P 12 8.55000019073) (P 12 10.0) 
(P 7 10.0) 
(P 20 10.0) 
(P 6 10.0) 

5046ee71bc77983a0753d6e5ba98f5f1c685072e
(P 5 8.63000011444) 

865981d5b40a693bafbadae4b1df769be03a25c3

9625e050ef2326c12632c51aea7b5e49a20d6fc7
(P 14 6.73000001907) (P 14 8.22000026703) (P 14 9.55000019073) 
(P 17 10.0)

In [18]:
# The goal is to concatenate small sub events into one big event
# For example Play/Pause/Play/Pause/Play/Pause on a video counts as only a Video event
def eventConcat(events):
    if len(events)<2:
        return events
    res=[events[0]]
    for event in events[1:-1]:
        if (
            event['EventType']!=res[-1]['EventType'] 
            #or event.get('VideoID',None)!=res[-1].get('VideoID',None)
            #or event.get('ProblemID',None)!=res[-1].get('ProblemID',None)
        ):
            res.append(event)
    res.append(events[-1])
    return res

def patternsConcat(patterns):
    res = {}
    for pattern in patterns.keys():
        res[pattern]=eventConcat(patterns[pattern])
    return res

In [19]:
rdd_events_by_students_extract_concat = (rdd_events_by_students_extract
    .map(lambda x: (x[0],patternsConcat(x[1])))
)

rdd_events_by_students_extract_concat.persist()

# Looks at the data for two students
displayStudentsPatterns(rdd_events_by_students_extract_concat,DEBUG_STUDENTS)

6ea6949ca133acede360d3573f9d1168b3d70b51
(P 14 4.5) (V 73) (P 14 6.88000011444) (P 14 10.0) 
(P 17 5.57000017166) 
(P 5 7.32999992371) (V 4) (P 5 10.0) 
(P 12 9.22999954224) 
(P 7 9.92000007629) 
(P 20 5.82999992371) (F) (V 125) (P 20 10.0) 
(P 6 9.67000007629) (P 6 10.0) 

67cdae6073d1089b695e2a615a01187586ad7ba6
(P 14 10.0) 
(P 17 10.0) 
(P 12 8.55000019073) (V 79) (P 14 10.0) (P 12 10.0) 
(P 7 10.0) 
(P 20 10.0) 
(P 6 10.0) 

5046ee71bc77983a0753d6e5ba98f5f1c685072e
(P 5 8.63000011444) 

865981d5b40a693bafbadae4b1df769be03a25c3

9625e050ef2326c12632c51aea7b5e49a20d6fc7
(P 14 6.73000001907) (P 14 9.55000019073) 
(P 17 10.0) 
(P 5 8.67000007629) (P 5 10.0) 
(P 12 10.0) 
(P 7 10.0) 
(P 20 8.93999958038) (P 20 10.0) 
(P 6 9.32999992371) 

3b305429f93de02637949578a5f9e23f13eb0726
(P 6 10.0) 
(P 7 10.0) 

fb4c81b3df430d1f0fbb8d0ca3e470ac6bf92a2f
(P 14 7.23000001907) (V 73) (P 14 8.40999984741) 
(P 17 0.0) (V 125) (P 17 7.01000022888) (F) (P 7 8.0) (F) (P 7 9.39999961853) (F) (P 17 7.44000

In [20]:
def eventToLetter(event):
    return {
        "Problem": lambda x:"P",
        "Video": lambda x:"V",
        "Forum": lambda x:"F",
    }[event['EventType']](event)

# Uses the method flatMap on the students patterns table to have a table of all the patterns
rdd_patterns = (rdd_events_by_students_extract_concat
    .flatMap(lambda x: [(x[0],pattern) for pattern in x[1].values()])
    # -> (studentId str, pattern eventList)
    .map(lambda x: (
        "".join([eventToLetter(event) for event in x[1]]),
        {
            "from":x[1][0]['Grade'],
            "to":x[1][-1]['Grade'],
            "id":x[1][0]['ProblemID'],
            "student":x[0]
        }
    ))
    .groupByKey()
)

rdd_patterns.persist()

# filters and shows the short (supposed most common) patterns 
for pattern in sorted(rdd_patterns.filter(lambda x: len(x[0])<6 and len(x[0])>2).take(100), key=(lambda x: len(x[0]))):
    print(pattern[0],len(pattern[1]))
    for student in list(pattern[1])[:1]:
        print("\t",student)
    print()

PFP 1133
	 {'from': '8.96000003815', 'id': '20', 'to': '10.0', 'student': 'c78a610dd94fd6b71665a2b6a7d236b6448ea2b5'}

PVP 2267
	 {'from': '8.67000007629', 'id': '5', 'to': '10.0', 'student': '7128643b0c9967d78bf21a2f062e280bc328a259'}

PFPP 363
	 {'from': '9.64999961853', 'id': '20', 'to': '5.82999992371', 'student': '9dcef769f4c3b8fc758904700e7a614c7aa32864'}

PVFP 73
	 {'from': '9.64999961853', 'id': '20', 'to': '10.0', 'student': 'af0f722c36ad0dded59bf27032ec414b4d7e6973'}

PVPP 1352
	 {'from': '4.5', 'id': '14', 'to': '10.0', 'student': '6ea6949ca133acede360d3573f9d1168b3d70b51'}

PFVP 73
	 {'from': '5.82999992371', 'id': '20', 'to': '10.0', 'student': '6ea6949ca133acede360d3573f9d1168b3d70b51'}

PVPFP 24
	 {'from': '7.21000003815', 'id': '17', 'to': '8.15999984741', 'student': '68a9f70b395697bde4205c406e4b9be601541008'}

PFPFP 244
	 {'from': '0.0', 'id': '6', 'to': '10.0', 'student': 'cba63263cf6f22245f80655bf0375b987b07a5c5'}

PVPVP 280
	 {'from': '8.67000007629', 'id': '5', 'to